In [1]:
import gym
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import os
import sys
sys.path.insert(1, './agent')
sys.path.insert(1, './price_pred')

import stock_env
from QLAgent import QNAgent
from helper import *

In [2]:
name_file_data = 'AAPL.csv'
name_project = 'AXP_batch'
env_name = 'StockEnv-v0'
num_episodes = 1000
len_obs = 100
len_window = 100
interval = 10
render_p = 100
init_balance = 1000
load_model = False
epoch_to_load = 10000
name_model_weights = 'trading_weights'
overlap = 20
batch_size = 1000
action_list=(-0.20, -0.10, 0, 0.10, 0.20)

In [11]:
path_models, path_stats = check_directories(name_project, len_obs, len_window)
np.random.seed(42)
# Read data you want to use for trading
train_data, test_data = get_data(f'./data/{name_file_data}')

In [12]:
# Create an instant
env = gym.make(env_name, train_data=train_data, eval_data=test_data, 
               len_obs=len_obs, len_window=len_window, init_balance=init_balance,
               action_list=action_list)
print(f'Observation space: {env.observation_space}')
print(f'Action space: {env.action_space}')

Observation space: Box(-5000.0, 5000.0, (100, 1), float32)
Action space: Discrete(5)


In [13]:
# Create an agent
agent = QNAgent(env, epsilon=1)

In [14]:
# Load model
if load_model:
    agent.load(path_models + name_model_weights + f'_{epoch_to_load}')
    train_statistics = pd.read_csv(path_stats + 'train.csv')
    test_statistics = pd.read_csv(path_stats + 'test.csv')
    init_ep = epoch_to_load
else:
    init_ep = 0
    train_statistics = pd.DataFrame()
    test_statistics = pd.DataFrame()

In [ ]:
worth = []
for ep in range(init_ep, num_episodes):
    get_performance(env, agent, train_data=True, training=True, batch_size=batch_size)
    if (ep % render_p) == 0:
        env.render(ep)
    worth.append(env.net_worth)
    if (ep % interval == 0) and not((load_model==True) and (ep == epoch_to_load)):
        agent.model.save_weights(path_models)

        overlap = overlap
        results_train = np.empty(shape=(0, 3))
        results_test = np.empty(shape=(0, 3))

        size_test = ((len(env.eval_data)-env.len_obs-env.len_window) // overlap)+1
        cagr_train, vol_train = get_performance(env, agent, train_data=True, training=False, batch_size=size_test)
        results_train = np.array([np.tile(ep, size_test), cagr_train, vol_train]).transpose()

        cagr_test, vol_test = get_performance(env, agent, train_data=False, training=False, overlap=overlap, batch_size=size_test)
        results_test = np.array([np.tile(ep, size_test), cagr_test, vol_test]).transpose()

        train_statistics = pd.concat([train_statistics, pd.DataFrame(results_train, columns=['epoch', 'cagr','volatility'])])
        train_statistics.to_csv(path_stats+'train.csv', index=False)
        test_statistics = pd.concat([test_statistics, pd.DataFrame(results_test, columns=['epoch', 'cagr','volatility'])])
        test_statistics.to_csv(path_stats+'test.csv', index=False)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


ep 0 
Profit: 4.1953650619%
Std: 0.17021134055487716

ep 100 
Profit: 4.8345930173%
Std: 0.17937147891496125

ep 200 
Profit: 4.793615828300001%
Std: 0.17037118880997

ep 300 
Profit: 5.5466530956%
Std: 0.18169478791342725

ep 400 
Profit: 4.7656700952%
Std: 0.1739756216940965


        plot_stocks_trading_performance(results_train, path_imgs + f'train_cagr_vol_ep_{ep}',
                                        color='royalblue', acc_title='Train')
        plot_stocks_trading_performance(results_test, path_imgs + f'test_cagr_vol_ep_{ep}',
                                        color='firebrick', acc_title='Test')

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.mean(worth,axis=-1))

In [ ]:
plot_stocks_trading_performance(train_statistics)

In [20]:
np.mean(worth)

1045.0000233226244

1049.890100331017